# setup

In [3]:
import torch
from transformers import *

import pandas as pd
pd.set_option('display.max_columns', 500)
import dask.dataframe as dd

HOME='/Users/yang.zhang/git/recsys20/'
p_in=f'{HOME}/data'

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# !head -1000000 {p_in}/training.tsv > {p_in}/trn1e6.tsv
# !head -100000 {p_in}/val.tsv > {p_in}/val1e5.tsv

    (rcss20) ➜  recsys20 git:(master) ✗ wc -l data/training.tsv
     148,075,238 data/training.tsv
    (rcss20) ➜  recsys20 git:(master) ✗ wc -l data/val.tsv
     15,127,684 data/val.tsv

# training.tsv

In [6]:
cols=[
'Text tokens',
'Hashtags',
'Tweet id',
'Present media',
'Present links',
'Present domains',
'Tweet type',
'Language',
'Timestamp',
    
'Engaged User id',
'Engaged Follower count',
'Engaged Following count',
'Engaged Is verified?',
'Engaged Account creation time',
    
'Engaging User id',
'Engaging Follower count',
'Engaging Following count',
'Engaging Is verified?',
'Engaging Account creation time',
    
'Engagee follows engager?',
'Reply engagement timestamp',
'Retweet engagement timestamp',
'Retweet with comment engagement timestamp',
'Like engagement timestamp',
]

cols_val=cols[:-4]

cols_time=[
'Timestamp',
'Engaged Account creation time',
'Engaging Account creation time',
'Reply engagement timestamp',
'Retweet engagement timestamp',
'Retweet with comment engagement timestamp',
'Like engagement timestamp',
]

In [7]:
# #https://docs.dask.org/en/latest/dataframe.html
# df=dd.read_csv(f'{p_in}/training10000.tsv',
#                sep='\x01',
#                header=None,
#                names=cols,)

df=pd.read_csv(f'{p_in}/trn1e6.tsv',
               sep='\x01',
               header=None,
               names=cols,)

In [8]:
df['did_rply']=~df['Reply engagement timestamp'].isna()
df['did_rtwt']=~df['Retweet engagement timestamp'].isna()
df['did_cmmt']=~df['Retweet with comment engagement timestamp'].isna()
df['did_like']=~df['Like engagement timestamp'].isna()

In [9]:
cols_tgt=['did_rply',
'did_rtwt',
'did_cmmt',
'did_like',]

In [11]:
pd.to_datetime(df[cols_time[0]].tail(), unit='s')

999995   2020-02-06 21:01:14
999996   2020-02-12 02:33:05
999997   2020-02-12 22:17:12
999998   2020-02-09 15:05:34
999999   2020-02-06 08:34:56
Name: Timestamp, dtype: datetime64[ns]

In [12]:
df['toks']=df['Text tokens'].apply(lambda x: x.split('\t'))

In [49]:
" ".join(tokenizer.convert_ids_to_tokens(df.sample().toks.values[0]))

'[CLS] [UNK] we augustus [UNK] them two [UNK] them 石 wiley [UNK] them 火 [UNK] [UNK] them 野 longing kits [UNK] [unused114] [UNK] ##dale contemporary 艹 [UNK] dull alternative le 葉 [UNK] cayman [UNK] found 西 ##ane ruin [unused114] [unused114] we augustus [UNK] [UNK] [UNK] approx [UNK] [unused114] [unused114] two [UNK] [UNK] parramatta ##manship [unused114] [unused114] 石 wiley [UNK] [UNK] [UNK] [unused114] [unused114] 火 [UNK] [UNK] [UNK] detached approx ##ン [unused114] [unused114] 野 longing kits [UNK] [UNK] [UNK] [UNK] [unused114] [unused114] carolina amateur but refrain dull relation politics ##van alternative 西 theorists [UNK] tasting 司 responding [unused126] [unused115] [unused115] [unused183] [unused114] playwright [unused115] [unused158] shouting succeeding [UNK] dec ##nched utc proposals [SEP]'

In [56]:
for col in [
'Present media',
'Present links',
'Present domains',
'Tweet type',
'Language',
'Engaged Follower count',
'Engaged Following count',
'Engaged Is verified?',
'Engaging Follower count',
'Engaging Following count',
'Engaging Is verified?',
'Engagee follows engager?',
]+cols_tgt:
    display(col, df[col].value_counts(dropna=False))

'Present media'

NaN                           620710
Photo                         182143
Video                         108825
Photo\tPhoto                   41294
Photo\tPhoto\tPhoto\tPhoto     23275
Photo\tPhoto\tPhoto            12932
GIF                            10800
Video\tVideo                      10
Photo\tVideo                       5
Video\tPhoto                       2
GIF\tGIF                           1
Video\tVideo\tVideo\tVideo         1
GIF\tVideo\tGIF                    1
Photo\tPhoto\tVideo                1
Name: Present media, dtype: int64

'Present links'

NaN                                 865464
E522B6C629B01DA6B29DE65BF5DFA55B       240
140BCA0056E50C79778369C95D6DDC63       103
DEA673FD63DDDC8A3606106D5CA235B0        98
270B33AB02B6FB3F60B748CF1CB9AB75        75
                                     ...  
D27BBC9FB4694DB876BCF8F178255CE4         1
B671C9BD954DB7B81DCC862E753C2E80         1
F9016E33938FF79EC7181A0DCE01EDD0         1
3EA3D9150B9E53E75818044B9A57188A         1
FB9DDD669590B104C7259A0CF9988EA7         1
Name: Present links, Length: 108173, dtype: int64

'Present domains'

...

False    886865
True     113135
Name: did_rtwt, dtype: int64

'did_cmmt'

False    992250
True       7750
Name: did_cmmt, dtype: int64

'did_like'

False    561196
True     438804
Name: did_like, dtype: int64

In [57]:
unq_twt_typs=df['Tweet type'].unique()

In [75]:
for col in cols_tgt:
    for typ in unq_twt_typs:
#         display(df[col][df['Tweet type']==typ].value_counts())
        print(col,typ,df[col][df['Tweet type']==typ].mean())

did_rply TopLevel 0.03858119741870459
did_rply Retweet 0.00758717775757398
did_rply Quote 0.034866451892978725
did_rtwt TopLevel 0.09875134734101376
did_rtwt Retweet 0.1425110713552512
did_rtwt Quote 0.09261544085726689
did_cmmt TopLevel 0.00818728390049974
did_cmmt Retweet 0.00673079743085902
did_cmmt Quote 0.00886512669362762
did_like TopLevel 0.5171024819071349
did_like Retweet 0.29684429715224214
did_like Quote 0.4805789750268467


In [78]:
print(df['Present media'].notna().mean())
for col in cols_tgt:
    print(col, df[col][df['Present media'].notna()].mean(), 
          df[col][df['Present media'].isna()].mean())

0.37929
did_rply 0.023061509662791004 0.03051505533985275
did_rtwt 0.12169052703735928 0.10790707415701374
did_cmmt 0.008434179651454033 0.007331926342414332
did_like 0.49185319939887684 0.4063878461761531


## val.tsv

In [62]:
dfval=dd.read_csv(f'{p_in}/val1e5.tsv',
                  sep='\x01',
               header=None,
               names=cols_val)

In [63]:
# %%time
# dfval.shape[0].compute()
# CPU times: user 2min 10s, sys: 29.3 s, total: 2min 39s
# Wall time: 1min 31s
# 15127684

In [64]:
dfval.head()

,Text tokens,Hashtags,Tweet id,Present media,Present links,Present domains,Tweet type,Language,Timestamp,Engaged User id,Engaged Follower count,Engaged Following count,Engaged Is verified?,Engaged Account creation time,Engaging User id,Engaging Follower count,Engaging Following count,Engaging Is verified?,Engaging Account creation time,Engagee follows engager?
0,101\t47185\t10157\t100986\t10343\t55422\t119\t...,NaN,7647B4E9DAF4C1D8973397DC2A04F3E3,Photo,NaN,NaN,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581703126,8A9AB92B775C62C4AB60DF6773A01571,13941,1216,False,1448292186,0000006C3074607050F1339DDCB890BB,27448,600,False,1520948869,True
1,101\t6006\t5086\t1939\t7418\t3601\t6406\t1913\...,NaN,CCBFBA5AFE7EFC03102EA8D0F86C4208,Photo,NaN,NaN,TopLevel,22C448FF81263D4BAF2A176145EE9EAD,1581736431,187AC59639DA9A6F32F7CD118EDD58F7,476439,1478,False,1254447722,00000776B07587ECA9717BFC301F2D6E,102,659,False,1478011810,False
2,101\t56898\t137\t44851\t10317\t11490\t10112\t1...,NaN,E18C2DCFC5AF20C650A0FD94598E69B7,Video,NaN,NaN,Retweet,ECED8A16BE2A5E8871FD55F4842F16B1,1582061925,82626B53CB2AD3B469E4AE06EAA9D930,367,702,False,1518708926,00000860E80C67D8C46CE57C64DE9444,230,189,False,1541013180,True
3,101\t13497\t10437\t94005\t11161\t73632\t11067\...,NaN,26DC813FDF8546B757BB9141099F119E,NaN,D58137F9D688C88435FD64FBAEA82B97,E91CDEC8DC7ABF30592FA024616FF970,TopLevel,ECED8A16BE2A5E8871FD55F4842F16B1,1582110043,7AFE06FF54898A1E9C716F539831849E,278,1229,False,1243548061,00000865A1538142CDA5936B07FE4311,65,165,False,1452599043,True
4,101\t24781\t10152\t42041\t38268\t10301\t10798\...,NaN,30A33055566AAC9EB18734C4EAD11FE1,NaN,AEF0CC9FA7B389B9A2ADF1331F00B65B,42DD9E2D4B2C0B0A71E909A6049EC2C2,TopLevel,D3164C7FBCF2565DDF915B1B3AEFB1DC,1581860270,D240DACE38CA84965270C86D47D3BF40,24313527,121,True,1177506290,00000865A1538142CDA5936B07FE4311,64,164,False,1452599043,False
